<a href="https://colab.research.google.com/github/DAVIDnHANG/DS-Unit-2-Regression-Classification/blob/master/module3/A3_regression_classification_TurnIN_W3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install category_encoders

     |████████████████████████████████| 102kB 2.7MB/s 


In [0]:
#imports here
import pandas as pd
import pandas_profiling
import datetime as dt
import numpy as np

In [0]:
# Read New York City property sales data
df = pd.read_csv('https://raw.githubusercontent.com/DAVIDnHANG/DS-Unit-2-Regression-Classification/master/data/condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int))


In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:
#restrict the data set between 100k to 2million of SALE_PRICE

#Turn the SALE PRICE, a object str, into a int.
df['SALE_PRICE'].astype(str).astype(int)
df['SALE_PRICE'] =df['SALE_PRICE'].replace(np.nan,0)
df=df.loc[(df['SALE_PRICE'] >= 100000) & (df['SALE_PRICE'] <= 2000000)]
df.head(3)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
65,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,691,1130,NaN,R4,"505 WEST 19TH STREET, 7D",7D,10011.0,1.0,0.0,1.0,"18,975",1077.0,2013.0,2,R4,2000000,01/02/2019


In [0]:
#restrict the date set further into only '01 ONE FAMILY DWELLINGS'
df=df.loc[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS')]
df.head(3)

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019


In [0]:
df.isnull().sum()


BOROUGH                              0
NEIGHBORHOOD                         0
BUILDING_CLASS_CATEGORY              0
TAX_CLASS_AT_PRESENT                 0
BLOCK                                0
LOT                                  0
EASE-MENT                         3164
BUILDING_CLASS_AT_PRESENT            0
ADDRESS                              0
APARTMENT_NUMBER                  3163
ZIP_CODE                             0
RESIDENTIAL_UNITS                    0
COMMERCIAL_UNITS                     0
TOTAL_UNITS                          0
LAND_SQUARE_FEET                     0
GROSS_SQUARE_FEET                    0
YEAR_BUILT                           0
TAX_CLASS_AT_TIME_OF_SALE            0
BUILDING_CLASS_AT_TIME_OF_SALE       0
SALE_PRICE                           0
SALE_DATE                            0
dtype: int64

In [0]:
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

# ```Date

let split the Data into two parts Jan 2019 - March 2019 and April 2019.
Use the three step

*   Convert to numpy, pandas format
*   Mask
*   then .loc all dates; in other words, filter.
Since we are training, testing, create test, train df





In [0]:
#SALE_DATE is in the format DD/MM/YYYY
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])
start_date = '01/01/2019'
end_date = '03/30/2019'
mask = (df['SALE_DATE'] > start_date) & (df['SALE_DATE'] <= end_date)
train_Date_March_April = df.loc[mask]
print(train_Date_March_April.shape)

df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])
start_date_test = '04/01/2019'
end_date_test = '04/30/2019'
mask_test = (df['SALE_DATE'] > start_date_test) & (df['SALE_DATE'] <= end_date_test)
test_April = df.loc[mask_test]
print(test_April.shape)
 


(2515, 21)
(606, 21)


# 1-hot encoding of categorical features
why do this?

*   Let say we have a catagory column which has index can be represented T and F.
*   So we can make those index on that column be turned into a new column which has 0 = F and F=1.
Now we can represents the categorical data.



In [0]:
#Before encoding let see how many columns a particular column would make.
#InThisArea=train_Date_March_April.groupby('BUILDING_CLASS_CATEGORY').sum()
#print(InThisArea)
TAX_CLASS=train_Date_March_April.groupby('EASE-MENT').sum()
print(TAX_CLASS)
#BLOCK=train_Date_March_April.groupby('BLOCK').sum()
#print(BLOCK)
#RESIDENTIAL_UNITS=train_Date_March_April.groupby('RESIDENTIAL_UNITS').sum()
#print(RESIDENTIAL_UNITS)
#BUILDING_CLASS_AT_TIME_OF_SALE = train_Date_March_April.groupby('BUILDING_CLASS_AT_TIME_OF_SALE').sum()
#print(BUILDING_CLASS_AT_TIME_OF_SALE)

Empty DataFrame
Columns: [BLOCK, LOT, ZIP_CODE, RESIDENTIAL_UNITS, COMMERCIAL_UNITS, TOTAL_UNITS, GROSS_SQUARE_FEET, YEAR_BUILT, TAX_CLASS_AT_TIME_OF_SALE, SALE_PRICE]
Index: []


# why show top?


1.   To articulate how many columns we need to build.
2.   If I "1-hot encode" blocks there will be 2065 new columns."
3.   1-hot encode building class will give me a dozen categoricals.
4.   so let encode tax_class at present because we will only be making less than 6 categoricals.



Before I can 1-hot encode there are some columns that I want to be left alone, but are object-string. need to convert them to floats.

In [0]:
#converting col Land SQ Feet into float64
train_Date_March_April['LAND_SQUARE_FEET'] = train_Date_March_April['LAND_SQUARE_FEET'].str.replace(',','').astype(np.float64)
test_April['LAND_SQUARE_FEET'] = test_April['LAND_SQUARE_FEET'].str.replace(',','').astype(np.float64)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
#let exclude catagory data with +6 cateogricals
target = 'SALE_PRICE'
high_cardiantlity = ['BOROUGH','NEIGHBORHOOD','BUILDING_CLASS_CATEGORY',
       'BLOCK','LOT' ,'LOT', 'EASE-MENT','BUILDING_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_PRESENT',
       'ADDRESS','ADDRESS','APARTMENT_NUMBER' ,'APARTMENT_NUMBER', 'ZIP_CODE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'ZIP_CODE', 'GROSS_SQUARE_FEET']
features = train_Date_March_April.columns.drop(high_cardiantlity + [target])
           
#train
X_train = train_Date_March_April[features]
y_train = train_Date_March_April[target]

#test
X_test = test_April[features]
y_test = test_April[target]

In [0]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
##train
X_train = encoder.fit_transform(X_train)

###Test
X_test = encoder.transform(X_test)

In [0]:
X_test.columns

Index(['TAX_CLASS_AT_PRESENT_1', 'TAX_CLASS_AT_PRESENT_1D',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'YEAR_BUILT', 'TAX_CLASS_AT_TIME_OF_SALE',
       'SALE_DATE'],
      dtype='object')

# Do feature selection with SelectKBest.

#we get to make engineer -features before we use SelectKBest, 
#@which randomly magically picked K features for us.

In [0]:
train_Date_March_April['GROSS_SQUARE_FEET']

Now what is important? in other words what is not important?
  
*   Borough Bronx, brooklyn, manhattan, queens
*   NEIGHBORHOOD - Lower manhattan, midtown manhattan, soho

*   BUILDING_CLASS_CATEGORY 

*  the whole dataframe can't be read because it either a string,or .dot


In [0]:
train_Date_March_April.columns

In [0]:
#train_Date_March_April
#test_April
def engineer_features(X):
  X = X.copy()
  #name all features
  name_all = ['RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       ]
  X['perk_count'] = X[name_all].sum(axis=1)
  return X
XUni_train = engineer_features(train_Date_March_April)
XUni_test = engineer_features(test_April)

In [0]:
drop = ['LOT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE', 'YEAR_BUILT','BUILDING_CLASS_CATEGORY','BLOCK','BOROUGH', 'NEIGHBORHOOD','TAX_CLASS_AT_PRESENT', 'EASE-MENT','BUILDING_CLASS_AT_PRESENT', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS','LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'TAX_CLASS_AT_TIME_OF_SALE','BUILDING_CLASS_AT_TIME_OF_SALE']
XUni_train=XUni_train.columns.drop(drop)
XUni_test=XUni_test.columns.drop(drop)

In [0]:
XUni_train

Index(['SALE_PRICE', 'SALE_DATE', 'perk_count'], dtype='object')

In [0]:
#will not be using feature eng for selector
#remove dates
from sklearn.feature_selection import f_regression, SelectKBest

#similar API to what we've seen before
selector = SelectKBest(score_func=f_regression,k=8)

#fit_transform on the train set
#.transform on test set
X_train_selected = selector.fit_transform(XUni_train, y_train)
X_test_selected = selector.fit_transform(XUni_test)
X_train_selected.shape, X_test_selected.shape

In [0]:
XUni_train.dtypes